In [1]:
import os
from mem0 import Memory
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

config = {
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "test_alice",
            "host": "localhost",
            "port": 6333,
        }
    }
}

m = Memory.from_config(config)
messages = [
    {"role": "user", "content": "I'm planning to watch a movie tonight. Any recommendations?"},
    {"role": "assistant", "content": "How about a thriller movies? They can be quite engaging."},
    {"role": "user", "content": "I’m not a big fan of thriller movies but I love sci-fi movies."},
    {"role": "assistant", "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future."}
]
m.add(messages, user_id="alice", metadata={"category": "movies"})

{'results': [{'id': 'd68a5327-b6b6-404d-a134-a5f6dcd472fc',
   'memory': 'Planning to watch a movie tonight',
   'event': 'ADD'},
  {'id': '43129e4e-1265-4b7d-9ee2-6241b1949cdb',
   'memory': 'Not a big fan of thriller movies',
   'event': 'ADD'},
  {'id': '1d63c0bd-0643-470a-bd82-0c27d0c1065e',
   'memory': 'Loves sci-fi movies',
   'event': 'ADD'}]}

In [36]:
a=m.get_all(user_id="alice")['results']

In [ ]:
import base64
from pathlib import Path
from typing import Optional

from agno.agent import Agent
from agno.media import Image
from agno.models.openai import OpenAIChat
from mem0 import MemoryClient

# Initialize the Mem0 client
client = MemoryClient()

# Define the agent
agent = Agent(
    name="Personal Agent",
    model=OpenAIChat(id="gpt-4"),
    description="You are a helpful personal agent that helps me with day to day activities."
                "You can process both text and images.",
    markdown=True
)


def chat_user(
    user_input: Optional[str] = None,
    user_id: str = "alex",
    image_path: Optional[str] = None
) -> str:
    """
    Handle user input with memory integration, supporting both text and images.

    Args:
        user_input: The user's text input
        user_id: Unique identifier for the user
        image_path: Path to an image file if provided

    Returns:
        The agent's response as a string
    """
    if image_path:
        # Convert image to base64
        with open(image_path, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("utf-8")

        # Create message objects for text and image
        messages = []

        if user_input:
            messages.append({
                "role": "user",
                "content": user_input
            })

        messages.append({
            "role": "user",
            "content": {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        })

        # Store messages in memory
        client.add(messages, user_id=user_id, output_format='v1.1')
        print("✅ Image and text stored in memory.")

    if user_input:
        # Search for relevant memories
        memories = client.search(user_input, user_id=user_id, output_format='v1.1')
        memory_context = "\n".join(f"- {m['memory']}" for m in memories['results'])

        # Construct the prompt
        prompt = f"""
You are a helpful personal assistant who helps users with their day-to-day activities and keeps track of everything.

Your task is to:
1. Analyze the given image (if present) and extract meaningful details to answer the user's question.
2. Use your past memory of the user to personalize your answer.
3. Combine the image content and memory to generate a helpful, context-aware response.

Here is what I remember about the user:
{memory_context}

User question:
{user_input}
"""
        # Get response from agent
        if image_path:
            response = agent.run(prompt, images=[Image(filepath=Path(image_path))])
        else:
            response = agent.run(prompt)

        # Store the interaction in memory
        interaction_message = [{"role": "user", "content": f"User: {user_input}\nAssistant: {response.content}"}]
        client.add(interaction_message, user_id=user_id, output_format='v1.1')
        return response.content

    return "No user input or image provided."

# Example Usage
if __name__ == "__main__":
    while True:
        user_input = input("🧑 You: ")
        if user_input.strip().lower() in {"exit", "quit"}:
            print("👋 Exiting agent session.")
            break
        else:
            response = chat_user(
                user_input,
                user_id="krish"
            )
            print("\n\nAGENT\n\n")
            print(response)

Hello Krish! How can I assist you with your tasks today? Do you need help with the development of the Fundraising and Governance Contracts, or you require support for setting up your Supabase project? Please provide me with clear details to ensure I can offer the most relevant help.
Sure, based on the information you previously provided, here is the plan for week 5:

- Krish will be setting up a Supabase project. 

Supabase is an open source Firebase alternative. He will specifically be dealing with the setup and initial configurations of this project. This might involve creating a new database, setting up authentication and rules, and planning the data structure.

- Furthermore, based on prior information, Krish will also be creating the following tables in Supabase:
  1. Users
  2. Projects
  3. Milestones
  4. Investments 

Frontend calls to Supabase will be utilized for reading and writing project data and user profiles.

This seems to be the highlight of his plans for week 5. It m

In [7]:
client.search("what is my name",user_id="alex")

AttributeError: 'NoneType' object has no attribute 'search'

In [9]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.mem0 import Mem0Tools
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["MEM0_API_KEY"] = os.getenv('MEM0_API_KEY')
agent = Agent(
    name="Memory Agent",
    user_id="krish",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[Mem0Tools()],
    description="An assistant that remembers and personalizes using Mem0 memory."
)

In [14]:
agent.run("""What is the plan of week 5""")

RunResponse(content='Could you please specify what context you\'re referring to for "week 5"? For example, is it related to a specific project, a study schedule, or something else?', content_type='str', thinking=None, reasoning_content=None, messages=[Message(role='system', content='An assistant that remembers and personalizes using Mem0 memory.', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, files=None, audio_output=None, image_output=None, thinking=None, redacted_thinking=None, provider_data=None, citations=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, audio_tokens=0, input_audio_tokens=0, output_audio_tokens=0, cached_tokens=0, cache_write_tokens=0, reasoning_tokens=0, prompt_tokens=0, completion_tokens=0, prompt_tokens_details=None, completion_tokens_details=None